## 提升训练速度的方法

1.对连接的权重采用一个更好的初始化策略  
2.使用一个更好的激活函数  
3.在训练时使用Batch Normalization  
4.复用一个已经预先训练过的神经网络  
5.使用一个相比梯度下降更快的优化器:  
e.g.,Momentum optimizaion, Nesterov Accelerated Gradient, AdaGrad, RMSProp, Adam optimization

**
目前最快的优化器是: Adam optimization  
Adam optimization通常是梯度下降速度的数倍  
Adam optimization没有一些可调的超参数(包括学习率)
**

**
Adam optimization的使用:  
只需要简单地将GradientDescentOptimizer替换为AdamOptimizer  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
**

## 通过正则化避免过拟合

### 1.Early Stopping

在验证集上的表现开始下降时停止训练  
在TensorFlow上的实现为:  
1.在固定的间隔(e.g.,50 steps)评估模型在验证集上的表现    
2.保存当前最优表现的快照  
3.在达到限定的步数时(e.g.,2000 steps),停止训练
4.计算最后的一个最优表现需要训练的步数

### 2. $l1,l2$正则化

In [ ]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [ ]:
scale = 0.001   #设置kernel_regularizer参数

In [ ]:
my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))   # 将activation,kernel_regularizer参数代入tf.layers.dense

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None,
                            name="outputs")

In [ ]:
with tf.name_scope("loss"):                                     # 定义损失命名空间
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  # 定义交叉熵
        labels=y, logits=logits)                                
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   # 定义基础损失函数
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)  # 定义正则化项
    loss = tf.add_n([base_loss] + reg_losses, name="loss")       # tf.add_n([p1, p2, p3....])函数是实现一个列表的元素的相加

In [ ]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

### 3.Dropout

最流行的深度学习正则化技术无疑是dropout

算法步骤:  
1.在每个训练step, 每个神经元都有一定的概率p被临时剔除 (包括输出层,但不包括输出层)  
2.在下一个step中, 这些神经元将被重新激活  
3.训练后,神经元将不再会被剔除
超参数p称为dropout rate, 通常设置为50%  
经过交叉验证，隐含节点dropout率等于0.5的时候效果最好，原因是0.5的时候dropout随机生成的网络结构最多。

一个重要的技术细节:  
假定P=50%, 那么在训练时,只有差不多一半的输入神经会出现  
因此,在测试和预测时,一个神经元将连接2倍于训练时输入神经的数量  
为了补偿这一情况, 需要在训练后, 对每一个输入神经元的连接权重乘以0.5  
一般地,在训练后,需要对每一个输入连接权重乘以一个 keep probability: $(1-p)$

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

需要在输入层和每一层隐含层中应用tf.layers.dropout函数

In [ ]:
training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)  # 使用的函数为tf.layers.dropout

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                              name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                              name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

In [ ]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={training: True, X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")

如果发现模型过拟合,应该增加dropout rate (即减少keep_prob)  
相反,如果发现模型欠拟合,则减小dropout rate  
同时,对于较大的层增加dropout rate; 对于较小的层减少dropout rate也很有用

dropout会显著地降低收敛的时间,但只要调节得当,通常会带来更好的模型  
对于额外的训练时间,使用该方法绝对是物有所值的

### 4.Max-Norm Regularization

对于每一个神经元,限制其连接的权重 $\left\| \mathbf{w} \right\|_2<=r$  
其中$r$为max-norm的超参数, 而 $\left\| \mathbf{·} \right\|_2$ 为 $l_2$ norm

1.在每一step后,计算$\left\| \mathbf{w} \right\|_2$  
2.在需要的情况下,降低 $\mathbf{w}$ $ \left( \mathbf{w} \gets \mathbf{w} \dfrac{r}{\left\| \mathbf{w} \right\|_2} \right) $

减少$r$值可以增加正则化效果,并消减过度拟合  
如果没有使用Batch Normalization, max-norm也能缓解vanishing/exploding gradients问题

### 5.Data Augmentation

e.g.,如果想要分类图片,可以对原始图片进行偏移,旋转和选择尺寸  
这种方法强迫模型对图片的位置,方向和尺寸有更大的容忍度  
同时,经过该方法,可以极大地增加训练集的数量

## 实践指导

**
初始化:  He initialization  
激活函数:  ELU  
标准化:  Batch Normalization  
正则化:  Dropout  
优化器(Optimizer):  Adam  
学习率schedule:  None
**

其他一些可作调整的默认设置:  
1.如果收敛过慢,则增加学习率; 如果收敛速度较快,但准确率不理性,则可以使用指数衰减的学习率schedule  
2.如果训练数据过少,则可以使用数据增强(Data Augmentation)  
3.如果想要得到一个稀疏的模型,可以使用$l_1$正则化,如果需要更稀疏的模型,可以将Adam optimization替换成FTRL(同时使用$l_1$)  
4.如果需要模型在运行期间有光速般的效率,可以去除Batch Normalization, 将ELU激活函数替换为leaky ReLU